## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Tresor II\r4401 Collins Ave PH-South\rMiami Beach,3/8/2023,177,"$14,000,000.00",4500,"$3,111.11",Linda Schechter,"Compass Florida, LLC.",Michele Redlich,Coldwell Banker Realty,Tresor II 4401 Collins Ave PH-South Miami Beach,Tresor II,4401 Collins Ave PH-South Miami Beach,"(25.8170079, -80.12229529999999)",25.817008,-80.122295
2,Tresor II\r4401 Collins Ave PH-North\rMiami Beach,3/8/2023,183,"$13,000,000.00",4500,"$2,888.89",Linda Schechter,"Compass Florida, LLC.",Michele Redlich,Coldwell Banker Realty,Tresor II 4401 Collins Ave PH-North Miami Beach,Tresor II,4401 Collins Ave PH-North Miami Beach,"(25.8165156, -80.1229681)",25.816516,-80.122968
3,Estates at Acqualina\r17901 Collins Ave 1704\r...,3/9/2023,170,"$8,100,000.00",4131,"$1,960.78",Yansy Checa,Acqualina Realty,Cynthia Kulonda,Southern REO South Florida Inc,Estates at Acqualina 17901 Collins Ave 1704 Su...,Estates at Acqualina,17901 Collins Ave 1704 Sunny Isles Beach,"(25.9420203, -80.1201787)",25.942020,-80.120179
4,Portofino Tower\r300 S Pointe Dr 2801\rMiami B...,3/9/2023,1,"$4,500,000.00",2340,"$1,923.08",John Lennon,South Pointe Drive Realty Inc,Prince Hassan,"Compass Florida, LLC.",Portofino Tower 300 S Pointe Dr 2801 Miami Beach,Portofino Tower,300 S Pointe Dr 2801 Miami Beach,"(25.7677055, -80.1348768)",25.767706,-80.134877
5,Murano Grande\r400 Alton Rd 1803\rMiami Beach,3/9/2023,15,"$4,375,000.00",3058,"$1,430.67",Francine Tomas,Brown Harris Stevens,Lisa Van Wagenen,Brown Harris Stevens,Murano Grande 400 Alton Rd 1803 Miami Beach,Murano Grande,400 Alton Rd 1803 Miami Beach,"(25.7731379, -80.14060429999999)",25.773138,-80.140604
6,Two Park Grove Condo\r2821 Bayshore 16C\rMiami,3/6/2023,89,"$3,225,000.00",2546,"$1,444.89",Ruedi Sieber,"Sieber International, LLC",Lorena Arnold,Keller Williams Capital Realty,Two Park Grove Condo 2821 Bayshore 16C Miami,Two Park Grove Condo,2821 Bayshore 16C Miami,"(25.7285074, -80.23808989999999)",25.728507,-80.238090
7,Ritz Carlton Residences\r15701 Collins Ave 260...,3/6/2023,34,"$2,730,550.00",1447,"$1,887.04",Cassio Galiza PA,Podium Realty Group INC,Cassio Galiza PA,Podium Realty Group INC,Ritz Carlton Residences 15701 Collins Ave 2603...,Ritz Carlton Residences,15701 Collins Ave 2603 Sunny Isles Beach,"(25.922045, -80.12199729999999)",25.922045,-80.121997
8,Murano Grande\r400 Alton Rd 1004\rMiami Beach,3/8/2023,317,"$2,350,000.00",1425,"$1,649.12",Ethan Pickett,Douglas Elliman,Leticia Leite,Elite International Realty Inc,Murano Grande 400 Alton Rd 1004 Miami Beach,Murano Grande,400 Alton Rd 1004 Miami Beach,"(25.7731379, -80.14060429999999)",25.773138,-80.140604
9,Marina Palms Residences\r17301 Biscayne Blvd 2...,3/6/2023,9,"$2,200,000.00",2390,$920.50,Rosita Cabrera PA,Coldwell Banker Realty,David Siddons,Douglas Elliman,Marina Palms Residences 17301 Biscayne Blvd 20...,Marina Palms Residences,17301 Biscayne Blvd 2010 Miami,"(25.9349574, -80.1503654)",25.934957,-80.150365
10,0Brickell Flatiron\r1000 Brickell Plaza 4502\r...,3/6/2023,6,"$2,175,000.00",1909,"$1,139.34",Javier Ortiz,Douglas Elliman,Giovanna Gonzalez,Florida Realty of Miami Corp,0Brickell Flatiron 1000 Brickell Plaza 4502 Miami,,0Brickell Flatiron 1000 Brickell Plaza 4502 Miami,"(25.7642644, -80.192725)",25.764264,-80.192725


In [9]:
df.at[10,'building_name']=('Brickell Flatiron')
df.at[10,'address_only']=('1000 Brickell Plaza 4502 Miami')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

## HTML Popup Formatter

In [14]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [18]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 5th - March 11th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [19]:
m.save('index.html')

## Data snagger

In [20]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [21]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Tresor II , 4401 Collins Ave PH-South Miami Beach | Price $14,000,000 | $3,111 psf | Listing agent: Linda Schechter with Compass Florida, LLC. | Buyer's agent: Michele Redlich with Coldwell Banker Realty | Days on market: 177
Least Expensive
Brickell Flatiron, 1000 Brickell Plaza 4502 Miami | Price $2,175,000 | $1,139 psf | Listing agent: Javier Ortiz with Douglas Elliman | Buyer's agent: Giovanna Gonzalez with Florida Realty of Miami Corp | Days on market: 6
Highest Price Per Square Foot
Tresor II , 4401 Collins Ave PH-South Miami Beach | Price $14,000,000 | $3,111 psf | Listing agent: Linda Schechter with Compass Florida, LLC. | Buyer's agent: Michele Redlich with Coldwell Banker Realty | Days on market: 177
Lowest Price Per Square Foot
Marina Palms Residences , 17301 Biscayne Blvd 2010 Miami | Price $2,200,000 | $920 psf | Listing agent: Rosita Cabrera PA with Coldwell Banker Realty | Buyer's agent: David Siddons with Douglas Elliman | Days on market: 9
Most Days on M

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [22]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [23]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_03132023
